In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
sparql = SPARQLWrapper("http://localhost:9999/blazegraph/namespace/hogeraad/sparql")

In [3]:
queryString = "prefix dcterm: <http://purl.org/dc/terms/> select (COUNT(*) as ?count) {?s dcterm:type <http://psi.rechtspraak.nl/uitspraak>}"
sparql.setQuery(queryString)
sparql.setReturnFormat(JSON)
ret = sparql.query()
ret.convert()

{'head': {'vars': ['count']},
 'results': {'bindings': [{'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '16389'}}]}}

In [4]:
searchstring = "werkgever aansprakelijkheid"

In [5]:
query_nodesandrefs = '''prefix dcterm: <http://purl.org/dc/terms/>
prefix bds: <http://www.bigdata.com/rdf/search#>
select ?type ?id ?to ?title ?creator ?date ?subject ?abstract
with {
 	select ?id
	where { 
      ?o bds:search "''' + searchstring + '''" .
      ?id ?p ?o .
      ?id dcterm:type	<http://psi.rechtspraak.nl/uitspraak>.
     }
} as %matcheddocs
where {
  {
    BIND("link" AS ?type).
    ?id dcterm:references ?to.
    include %matcheddocs
  }
   union
    {
    include %matcheddocs
    BIND("node" AS ?type).
   ?id dcterm:type	<http://psi.rechtspraak.nl/uitspraak>.
       optional { ?id dcterm:creator ?creator}.
   optional { ?id dcterm:abstract ?abstract}.
   optional { ?id dcterm:subject ?subject}.
   optional { ?id dcterm:date ?date}.
   optional { ?id dcterm:title ?title}
  }
}
'''
sparql.setQuery(query_nodesandrefs)
sparql.setReturnFormat(JSON)
ret = sparql.query()
nodesandrefs = ret.convert()

In [6]:
nodes = [res for res in nodesandrefs['results']['bindings'] if res['type']['value']=='node']
links = [res for res in nodesandrefs['results']['bindings'] if res['type']['value']=='link']
print(len(nodes), len(links))

865 286


In [7]:
nodesandrefs['head']['vars']

['type', 'id', 'to', 'title', 'creator', 'date', 'subject', 'abstract']

In [8]:
def nodes_to_json(nodes_in, variables):
    nodes_json = []
    unique_ids = []
    for d in nodes_in:
        if d['id']['value'] not in unique_ids:
            dout = {}
            for var in variables:
                dout[var] = d.get(var,{'value':''})['value']
            nodes_json.append(dout)
            unique_ids.append(d['id']['value'] )
    return nodes_json, unique_ids

In [9]:
variables = [x for x in nodesandrefs['head']['vars'] if x not in ['type', 'from', 'to']]
nodes_json, unique_ids = nodes_to_json(nodes, variables)

In [10]:
nodes_json[0]

{'abstract': 'Faillissementsrecht. Bestuurdersaansprakelijkheid boedeltekort; kennelijk onbehoorlijke bestuur; hoofdelijke aansprakelijkheid van een rechtspersoon als bestuurder van een andere rechtspersoon als bedoeld in art. 2:11 BW (81 RO).',
 'creator': 'http://standaarden.overheid.nl/owms/terms/Hoge_Raad_der_Nederlanden',
 'date': '2008-12-19',
 'id': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2008:BG3579',
 'subject': 'http://psi.rechtspraak.nl/rechtsgebied#civielRecht',
 'title': 'ECLI:NL:HR:2008:BG3579 Hoge Raad , 19-12-2008 / C07/120HR'}

In [11]:
len(links)

286

In [12]:
#
def links_to_json(links_in, node_ids):
    links_json = []
    for d in links_in:
        target = d['to']['value']
        if target in node_ids:
            dout = {'source': d['id']['value'],
                    'target': target }
            dout['id'] = dout
            links_json.append(dout)
    return links_json

In [13]:
references = links_to_json(links, unique_ids)

In [14]:
ids_ref = set([d['source'] for d in references] + [d['target'] for d in references])
ids_nodes = set([d['id'] for d in nodes_json])

In [15]:
print(len(ids_ref), len(links), len(ids_nodes), len(nodes))

87 286 705 865


In [46]:
import json
filename = '/media/sf_VBox_Shared/CaseLaw/graphs/d3.json'

with open(filenaam, 'w') as outfile:
    json.dump({'nodes': nodes_json, 'links': references}, fp=outfile)

In [18]:
import json

filename = '/media/sf_VBox_Shared/CaseLaw/graphs/sigma.json'
#Set id for sigma
for r in references:
    r['id'] = r['source'] + '_' + r['target']

with open(filename, 'w') as outfile:
    json.dump({'nodes': nodes_json, 'edges': references}, fp=outfile)

In [21]:
sum([node['date']=='' for node in nodes_json])

0